In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import re
import requests
from random import *
from tqdm import tqdm #progress bar
from datetime import datetime
import os
import glob


In [2]:
# concatenate the dataframes in one folder to get one file (with different columns)
def concatenate_dfs(indir, save_to_csv = True, save_to_pickle = True):
    

    fileList=glob.glob(str(str(indir) + "*.csv"))
    print("Found this many CSVs: ", len(fileList), " In this folder: ", str(os.getcwd()) + "/" + str(indir))

    output_file = pd.concat([pd.read_csv(filename) for filename in fileList])

    if save_to_csv:
        output_file.to_csv("data/de/make_model_ads_concatinated.csv", index=False)

    if save_to_pickle:
        output_file.to_pickle("data/de/make_model_ads_concatinated.pkl")

    output_file.reset_index(drop=True, inplace=True)

    return(output_file)

def clean_data(data):
    data['price'] = data['Preis'].str.replace('.', '')
    data['price'] = data['price'].str.extract('(\d+)').astype(int, errors='ignore')

    data = data[~data['price'].isna()]
    data['price'] = data['price'].astype(int)

    return data

def merge_make_model_keep_latest(data):
    latest_scrape = data.groupby(['link'], dropna=True).agg(number_of_ads=('link', 'count'), latest_scrape=('download_date_time', 'max'))
    latest_scrape = latest_scrape.reset_index()

    data = pd.merge(data, latest_scrape[['link', 'latest_scrape']], how = 'left', left_on = ['link'], right_on = ['link'])
    data = data.reset_index(drop=True)
    # keep rows where download_date_time is equal to latest_scrape
    data = data[data['download_date_time'] == data['latest_scrape']]
    data = data.reset_index(drop=True)
    # drop the latest_scrape column
    data = data.drop(columns = ['latest_scrape'])
    
    data = pd.merge(data, make_model_ads_data_latest[['ad_link', 'car_make', 'car_model']], how = 'left', left_on = 'link', right_on = 'ad_link')

    return data



In [7]:
individual_ads_data = concatenate_dfs("data/de/make_model_ads_data/",  True, True)



Found this many CSVs:  72  In this folder:  /Users/josemariamoyano/Documents/python_projects/car_cormparator/notebooks/data/de/make_model_ads_data/


In [8]:
individual_ads_data

,Preis,Kategorie,Kilometerstand,Hubraum,Leistung,Kraftstoffart,Anzahl Sitzplätze,Getriebe,Erstzulassung,Farbe,Innenausstattung,link,download_date_time
0,11.990 €Guter PreisZur Preisbewertung,Kombi,128.138 km,1.968 cm³,110 kW (150 PS),Diesel,5.0,Schaltgetriebe,09/2014,Silber Metallic,Schwarz,https://suchen.mobile.de/fahrzeuge/details.htm...,20220413_185610
1,10.400 €Ohne BewertungKeine realistische Preis...,Limousine,90.939 km,1.197 cm³,77 kW (105 PS),Benzin,5.0,Automatik,02/2012,Schwarz,"Stoff, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220413_185407
2,35.490 €Guter PreisZur Preisbewertung,"SUV / Geländewagen / Pickup, Tageszulassung",10 km,1.498 cm³,110 kW (150 PS),Benzin,5.0,Automatik,12/2021,Grau Metallic,"Stoff, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220413_184902
3,19.590 €Fairer PreisZur Preisbewertung,SUV / Geländewagen / Pickup,108.391 km,1.498 cm³,110 kW (150 PS),Benzin,5.0,Schaltgetriebe,03/2018,Blau Metallic,"Stoff, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220413_185215
4,24.990 €Fairer PreisZur Preisbewertung,SUV / Geländewagen / Pickup,50.635 km,1.498 cm³,110 kW (150 PS),Benzin,5.0,Automatik,08/2018,Rot,"Stoff, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220413_185003
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,16.990 €Guter PreisZur Preisbewertung,SUV / Geländewagen / Pickup,108.589 km,1.968 cm³,103 kW (140 PS),Diesel,5.0,Schaltgetriebe,03/2015,Weiß,"Stoff, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220413_185740
68,29.970 €Erhöhter PreisZur Preisbewertung,SUV / Geländewagen / Pickup,80.000 km,1.968 cm³,110 kW (150 PS),Diesel,5.0,Automatik,03/2018,Grau,"Velours, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220413_184933
69,11.900 €Guter PreisZur Preisbewertung,Kombi,90.283 km,1.197 cm³,81 kW (110 PS),Benzin,5.0,Schaltgetriebe,04/2016,Grau Metallic,"Stoff, Grau",https://suchen.mobile.de/fahrzeuge/details.htm...,20220413_185620
70,15.800 €Fairer PreisZur Preisbewertung,SUV / Geländewagen / Pickup,140.122 km,1.968 cm³,103 kW (140 PS),Diesel,5.0,Automatik,03/2015,Braun Metallic,"Stoff, Beige",https://suchen.mobile.de/fahrzeuge/details.htm...,20220413_190006


In [10]:
ads_df_clean = clean_data(data = individual_ads_data)

ads_df_clean.columns = ['Price', 'Category', 'Mileage', 'Engine_displacement', 'Power',
       'fuel_type', 'number_of_seats', 'transmission', 'first_registration',
       'color', 'interior', 'link', 'download_date_time', 'price']
ads_df_clean

/var/folders/k8/kj3chpcj10b9fvt2m3vmgzbw0000gn/T/ipykernel_84044/1883972105.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['price'] = data['Preis'].str.replace('.', '')
/var/folders/k8/kj3chpcj10b9fvt2m3vmgzbw0000gn/T/ipykernel_84044/1883972105.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['price'].astype(int)


,Price,Category,Mileage,Engine_displacement,Power,fuel_type,number_of_seats,transmission,first_registration,color,interior,link,download_date_time,price
0,11.990 €Guter PreisZur Preisbewertung,Kombi,128.138 km,1.968 cm³,110 kW (150 PS),Diesel,5.0,Schaltgetriebe,09/2014,Silber Metallic,Schwarz,https://suchen.mobile.de/fahrzeuge/details.htm...,20220413_185610,11990
1,10.400 €Ohne BewertungKeine realistische Preis...,Limousine,90.939 km,1.197 cm³,77 kW (105 PS),Benzin,5.0,Automatik,02/2012,Schwarz,"Stoff, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220413_185407,10400
2,35.490 €Guter PreisZur Preisbewertung,"SUV / Geländewagen / Pickup, Tageszulassung",10 km,1.498 cm³,110 kW (150 PS),Benzin,5.0,Automatik,12/2021,Grau Metallic,"Stoff, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220413_184902,35490
3,19.590 €Fairer PreisZur Preisbewertung,SUV / Geländewagen / Pickup,108.391 km,1.498 cm³,110 kW (150 PS),Benzin,5.0,Schaltgetriebe,03/2018,Blau Metallic,"Stoff, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220413_185215,19590
4,24.990 €Fairer PreisZur Preisbewertung,SUV / Geländewagen / Pickup,50.635 km,1.498 cm³,110 kW (150 PS),Benzin,5.0,Automatik,08/2018,Rot,"Stoff, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220413_185003,24990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,16.990 €Guter PreisZur Preisbewertung,SUV / Geländewagen / Pickup,108.589 km,1.968 cm³,103 kW (140 PS),Diesel,5.0,Schaltgetriebe,03/2015,Weiß,"Stoff, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220413_185740,16990
68,29.970 €Erhöhter PreisZur Preisbewertung,SUV / Geländewagen / Pickup,80.000 km,1.968 cm³,110 kW (150 PS),Diesel,5.0,Automatik,03/2018,Grau,"Velours, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220413_184933,29970
69,11.900 €Guter PreisZur Preisbewertung,Kombi,90.283 km,1.197 cm³,81 kW (110 PS),Benzin,5.0,Schaltgetriebe,04/2016,Grau Metallic,"Stoff, Grau",https://suchen.mobile.de/fahrzeuge/details.htm...,20220413_185620,11900
70,15.800 €Fairer PreisZur Preisbewertung,SUV / Geländewagen / Pickup,140.122 km,1.968 cm³,103 kW (140 PS),Diesel,5.0,Automatik,03/2015,Braun Metallic,"Stoff, Beige",https://suchen.mobile.de/fahrzeuge/details.htm...,20220413_190006,15800


In [15]:
ads_df_clean.loc[5]['link']

'https://suchen.mobile.de/fahrzeuge/details.html?id=334430988&damageUnrepaired=NO_DAMAGE_UNREPAIRED&isSearchRequest=true&makeModelVariant1.makeId=25200&makeModelVariant1.modelId=54&pageNumber=1&ref=quickSearch&scopeId=C&sfmr=false&searchId=86c4b254-2f56-b938-9800-44d91497c51b'

In [16]:
def concatenate_dfs(indir, save_to_csv = True, save_to_pickle = True):
    

    fileList=glob.glob(str(str(indir) + "*.csv"))
    print("Found this many CSVs: ", len(fileList), " In this folder: ", str(os.getcwd()) + "/" + str(indir))

    output_file = pd.concat([pd.read_csv(filename) for filename in fileList])

    if save_to_csv:
        output_file.to_csv("data/de/make_model_ads_links_concatinated.csv", index=False)

    if save_to_pickle:
        output_file.to_pickle("data/de/make_model_ads_links_concatinated.pkl")

    return(output_file)




make_model_ads_data = concatenate_dfs(indir= "data/de/make_model_ads_links/", save_to_csv = False, save_to_pickle = True)

Found this many CSVs:  10  In this folder:  /Users/josemariamoyano/Documents/python_projects/car_cormparator/notebooks/data/de/make_model_ads_links/


In [18]:
make_model_ads_data['car_model'].unique()

array(['Golf', 'T-Roc', 'X1', 'Tiguan', 'V60', 'A3'], dtype=object)